In [76]:
import json
 
with open("C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/articles_ytn_tokenized.json") as json_file:
    data = json.load(json_file)

data['sentence_tokens'] = []
data['sentence_pos'] = []

In [67]:
import re

# Our Analysis Variables:

- Length of the string 텍스트 스트링의 길이 (for equal comparison)
- Number of tokens 토큰 수
- Number of sentences 문장 수
- Tokens sorted by frequency 빈도 수로 정렬한 토큰 리스트
- Sentence structure category sorted by frequency 빈도 수로 정렬한 문장 구조 카테고리

## Length of the string 텍스트 스트링의 길이 (for equal comparison)

In [68]:
def get_string_length(text):
    return(len(text))

## Number of tokens 토큰 수

In [69]:
def get_num_tokens(tokens):
    return(len(tokens))

## Number of sentences 문장 수

In [70]:
def find_sent_id(tokens):
    ids = []
    for i, tok in enumerate(tokens):
        found = re.findall('[.?!]+', tok)
        if(bool(found)):
            ids.append(i)
    return ids

def split_sentence(sent_ids, tokens):  
    sentences = []
    start = 0
    for i in sent_ids:
        sentences.append(tokens[start:i+1])
        start = i+1
    return sentences

In [71]:
def get_num_sentences(tokens):
    sent_ids = find_sent_id(tokens)
    sentences = split_sentence(sent_ids, tokens)
    return (len(sent_ids), sentences)

In [78]:
all_analysis = {'string_length':[], 'num_tokens':[], 'num_sentences':[]}
def simple_analysis(data):
    for i in range(len(data['text'])):
        text = data['text'][i]
        tokens = data['tokens'][i]
        all_analysis['string_length'].append(get_string_length(text))
        all_analysis['num_tokens'].append(get_num_tokens(tokens))
        num_sentences, sentences = get_num_sentences(tokens)
        all_analysis['num_sentences'].append(num_sentences)
        data['sentence_tokens'].append(sentences)        

In [79]:
simple_analysis(data)
print(all_analysis)


{'string_length': [603, 830, 299, 523, 803, 298, 385, 8861, 8502, 7693, 1920, 304, 1206, 930, 608, 396, 530, 479, 710, 737, 317, 457, 353, 435, 4091, 437, 352, 147, 365, 892, 1208, 191, 414, 358, 286, 587, 430, 277, 402, 264, 242, 744, 467, 875, 554, 711, 368, 586, 413, 436, 282, 182, 199, 485, 188, 2427, 215, 1113, 1298, 1160, 1057, 386, 458, 858, 152, 933, 409, 850, 1155, 1313, 152, 457, 850, 1313, 1155, 254, 242, 237, 524, 1210, 412, 1714, 409, 933, 11081, 1533, 326, 376, 919, 455, 822, 121, 738, 259, 921, 1571, 454, 481, 197, 468, 1115, 858, 1019, 1213, 379, 484, 346, 428, 1046, 1115, 320, 154, 1044, 626, 982, 154, 230, 348, 134, 1044, 1244, 982, 626, 484, 1367, 189, 413, 742, 204, 1367, 1347, 346, 320, 170, 599, 150, 241, 433, 487, 428, 275, 1115, 540, 232, 1400, 1354, 232, 462, 262, 1046, 859, 938, 1006, 463, 460, 628, 430, 2667, 170, 874, 313, 773, 1088, 680, 502, 1266, 1137, 820, 680, 680, 155, 1005, 1094, 462, 462, 742, 5479, 2686, 2686, 413, 240, 517, 892, 582, 773, 740, 333,

In [82]:
print(len(all_analysis['string_length']))

6667


## Tokens sorted by frequency 빈도 수로 정렬한 토큰 리스트

In [86]:
from collections import Counter
all_tokens = {}
def get_token_dist(tokens, all_tokens):
    micro_dict = dict(Counter(tokens).most_common(len(tokens)))
    for tok, num in micro_dict.items():
        if(tok in all_tokens):
            all_tokens[tok] += num
        else:
            all_tokens.update({tok: num})

In [90]:
for i in range(len(data['tokens'])):
    tokens = data['tokens'][i]
    all_tokens = {}
    get_token_dist(tokens, all_tokens)

In [92]:
print(all_tokens)

{'이': 29, '하': 25, '.': 17, '었': 15, '우': 13, 'ㄴ': 13, '습니다': 13, '크': 12, '라': 12, '나': 12, '에': 11, '를': 11, '지원': 8, '고': 8, '어': 8, '되': 8, '의': 8, '는': 8, '독일': 7, '을': 6, '은': 6, '전쟁': 5, '와': 5, ',': 5, '레오파드': 5, '가': 5, '탱크': 4, '부패': 4, '등': 4, 'ㅂ니다': 4, '2': 4, '도': 4, '러시아': 4, 'ㄹ': 4, '있': 4, '결정': 3, '5': 3, '개': 3, '주지사': 3, '로': 3, '자': 3, '다고': 3, '과': 3, '보도': 3, '기로': 2, '것': 2, '으로': 2, '지': 2, '와중': 2, '국방': 2, '차관': 2, '검찰': 2, '부총장': 2, '고위직': 2, '대거': 2, '물갈이': 2, '국가': 2, '필요': 2, '수': 2, '들': 2, '조만간': 2, '겠': 2, '/': 2, ':': 2, '대하': 2, '총리': 2, '키': 2, '에서': 2, '함께': 2, '국방부': 2, '반': 2, '위하': 2, '놓': 1, '주저': 1, '오': 1, '결국': 1, '전하': 1, '의혹': 1, '제기': 1, '면서': 1, '종수': 1, '기자': 1, '기술': 1, '집대성': 1, '유럽': 1, '다수': 1, '운용': 1, '달라': 1, '일찌감치': 1, '요청': 1, '다르': 1, '나라': 1, '제공': 1, '려면': 1, '승인': 1, '자극': 1, '확대': 1, '잇닿': 1, '우려': 1, '소극적': 1, '태도': 1, '보이': 1, '이에': 1, '북': 1, '대서양': 1, '조약': 1, '기구': 1, '나토': 1, '회원국': 1, '압박': 1, '심하': 1, '여부': 1, '답': 

## Sentence structure category sorted by frequency 빈도 수로 정렬한 문장 구조 카테고리